In [1]:
## One Hot Encoding
from sklearn.preprocessing import LabelEncoder
## train test split
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [32]:
df = pd.read_csv('merged_data.csv')

In [33]:
df.shape

(121838, 32)

In [34]:
df['seq'] = df['sequence'].str[1:6]

In [35]:
df.head()

,transcript_id,transcript_position,sequence,avg_1-flank_length,avg_1-flank_std,avg_1-flank_mean,avg_central_length,avg_central_std,avg_central_mean,avg_1+flank_length,...,std_1-flank_mean,std_central_length,std_central_std,std_central_mean,std_1+flank_length,std_1+flank_std,std_1+flank_mean,gene_id,label,seq
0,ENST00000000233,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,...,4.944365,0.005364,3.302671,2.765244,0.004566,2.906225,2.522168,ENSG00000004059,0,AGACC
1,ENST00000000233,261,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,...,2.981182,0.003767,1.237045,3.526528,0.004528,1.466295,2.492530,ENSG00000004059,0,AAACT
2,ENST00000000233,316,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,...,1.492577,0.005035,0.893481,3.203816,0.004497,0.800496,1.484341,ENSG00000004059,0,AAACA
3,ENST00000000233,332,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006102,...,3.202963,0.004612,1.393868,1.928009,0.003595,0.741771,3.494937,ENSG00000004059,0,GAACA
4,ENST00000000233,368,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,...,2.664643,0.005407,1.753894,2.177236,0.005908,1.341822,2.379045,ENSG00000004059,0,GGACA


In [36]:
df_le = df.copy()
holder = df_le["seq"].values.reshape(-1, 1)

label_encoder = LabelEncoder()

temp = pd.DataFrame(label_encoder.fit_transform(holder))

temp.columns = ["seq_le"]

df_le = df_le.join(temp)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [37]:
df_le = df_le.drop(["sequence", "seq"], axis=1)

In [38]:
df_le.columns

Index(['transcript_id', 'transcript_position', 'avg_1-flank_length',
       'avg_1-flank_std', 'avg_1-flank_mean', 'avg_central_length',
       'avg_central_std', 'avg_central_mean', 'avg_1+flank_length',
       'avg_1+flank_std', 'avg_1+flank_mean', 'med_1-flank_length',
       'med_1-flank_std', 'med_1-flank_mean', 'med_central_length',
       'med_central_std', 'med_central_mean', 'med_1+flank_length',
       'med_1+flank_std', 'med_1+flank_mean', 'std_1-flank_length',
       'std_1-flank_std', 'std_1-flank_mean', 'std_central_length',
       'std_central_std', 'std_central_mean', 'std_1+flank_length',
       'std_1+flank_std', 'std_1+flank_mean', 'gene_id', 'label', 'seq_le'],
      dtype='object')

In [39]:
train, test = train_test_split(df_le, test_size=0.2)

In [40]:
train.shape

(97470, 32)

In [41]:
test.shape

(24368, 32)

In [42]:
len(train[train['label'] == 0])

93082

In [43]:
len(train[train['label'] == 1])

4388

In [64]:
1 - 4332/93138

0.9534883720930233

In [58]:
X_train = train.drop(['transcript_id', 'gene_id', 'label'], axis=1)
y_train = train['label']

X_test = test.drop(['transcript_id', 'gene_id', 'label'], axis=1)
y_test = test['label']

In [59]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampledtrain, y_resampledtrain = smote.fit_resample(X_train, y_train)
X_resampledtest, y_resampledtest = smote.fit_resample(X_test, y_test)


In [60]:
original_class_counts = y_train.value_counts()
resampled_class_counts = pd.Series(y_resampledtrain).value_counts()

print("Original Class Distribution:")
print(original_class_counts)

print("\nResampled Class Distribution:")
print(resampled_class_counts)

Original Class Distribution:
label
0    93082
1     4388
Name: count, dtype: int64

Resampled Class Distribution:
label
0    93082
1    93082
Name: count, dtype: int64


In [61]:
original_class_counts = y_test.value_counts()
resampled_class_counts = pd.Series(y_resampledtest).value_counts()

print("Original Class Distribution:")
print(original_class_counts)

print("\nResampled Class Distribution:")
print(resampled_class_counts)

Original Class Distribution:
label
0    23281
1     1087
Name: count, dtype: int64

Resampled Class Distribution:
label
0    23281
1    23281
Name: count, dtype: int64


In [56]:
resampled_train = pd.DataFrame(data=X_resampledtrain, columns=X_train.columns)
resampled_train['label'] = y_resampledtrain
resampled_train.to_csv('resampled_traindata.csv', index=False)


In [62]:
resampled_test = pd.DataFrame(data=X_resampledtest, columns=X_test.columns)
resampled_test['label'] = y_resampledtest
resampled_test.to_csv('resampled_testdata.csv', index=False)